In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
raw_data = pd.read_csv('datasets/samsung200129_220328.csv')
print(raw_data.head())
raw_data.info()

In [ ]:
raw_data['Date'] = pd.to_datetime(raw_data['Date'])
raw_data.set_index('Date', inplace=True)
print(raw_data.head())

In [ ]:
raw_data['Close'].plot()
plt.show()

In [ ]:
data_test = raw_data.sort_values('Close')
print(data_test.head())
print(data_test.tail())

In [ ]:
data_close = raw_data[['Close']]
print(data_close.head())

In [ ]:
minmaxscaler = MinMaxScaler()
scaled_data = minmaxscaler.fit_transform(data_close)
print(scaled_data[:5])
print(scaled_data.shape)

In [ ]:
sequence_X = []
sequence_Y = []

for i in range(len(scaled_data)-30):
    x = scaled_data[i:i+30]
    y = scaled_data[i+30]
    sequence_X.append(x)
    sequence_Y.append(y)
print(sequence_X[:5])
print(sequence_Y[:5])

In [ ]:
sequence_X = np.array(sequence_X)
sequence_Y = np.array(sequence_Y)
print(sequence_X[0])
print(sequence_Y[0])
print(sequence_X.shape)
print(sequence_Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(sequence_X, sequence_Y, test_size=0.2)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(30, 1), activation='tanh'))  # lstm은 활성함수 탄젠트 국룰
model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.summary()


In [ ]:
fit_hist = model.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test), shuffle=False)

In [ ]:
plt.plot(fit_hist.history['loss'][10:], label='loss')
plt.plot(fit_hist.history['val_loss'][10:], label='val_loss')
plt.legend()
plt.show()

In [ ]:
model.save('./stock_close_predict.h5')

In [ ]:
pred = model.predict(X_test)

In [ ]:
plt.plot(Y_test[:30], label='actual')
plt.plot(pred[:30], label='predict')
plt.legend()
plt.show()

In [ ]:
last_data_30 = scaled_data[-30:].reshape(1, 30, 1)
today_close = model.predict(last_data_30)

print(today_close)

In [ ]:
today_close_value = minmaxscaler.inverse_transform(
    today_close)
print(today_close_value)